In [32]:
if(!dir.exists("Functions/")){
    setwd("../")
    if(!dir.exists("Functions")){
        setwd("M:/lecospec/lecospec/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)

Loading required package: rasterVis



In [33]:
log_model_results <- function(model_id, confusion_matrix, distribition, custom = NULL, logpath = "./gs.log"){
    # append performance data to the logs for later comparison
    sink(file = logpath, append = TRUE)
    print("-------------------------------------------------------")
    print("---------------------- Model Data ---------------------")
    
    print(paste0("Model Type: PLS-LDA (plsgenomics)"))
    print(paste0("Data Index: ",custom))
    print(paste0("Model UUID: ", model_id))
    print("---------------------- Confusion Matrix ---------------------")
    print(confusion_matrix)
    print("---------------------- Class Distribution ---------------------")
    print(distribition)
    print("-------------------------------------------------------")
    sink(NULL)
}

In [34]:
add_model_to_manifest <- function(
    model_id, 
    outlier = "", 
    preprocessing="",
    source="", 
    weight = "",
    n = "",
    oob_error = "",
    accuracy = "",
    r2 = "",
    chi2prob = "",
    logpath="./gs_manifest.csv"){
    if(!file.exists(logpath)){
        header <- "source,outliers,preprocessing,weight,n,oob,accuracy,r2,chi2prob,model_id"
        write(header, file = logpath)
    }

    line <- paste(
        source,
        outlier,
        preprocessing,
        weight,
        n,
        oob_error,
        accuracy,
        r2,
        chi2prob,
        sep=","
    )
    line <- paste0(line, ",", model_id)

    write(line, file=logpath, append = TRUE)
}

In [35]:
train_pls_lda <- function(
    train_df, 
    train_labels,
    test_df, 
    test_labels,
    n = 32,
    outlier_fn = NULL,
    preprocess_fn = NULL,
    weight_fn = targets_to_weights,
    model_id = uuid::UUIDgenerate(),
    ignore_cols = NULL,
    save_path = "./mle/models/gs/",
    seed = NULL,
    log_string = ""
){

    if(!is.null(seed)){
        set.seed(seed)
    }

    x_train <- train_df %>% as.data.frame()
    x_test <- test_df %>% as.data.frame()
    if(is.function(outlier_fn)){
        x_train <- outlier_fn(x_train)
    }
    if(is.function(preprocess_fn)){
        x_train <- preprocess_fn(x_train)
        x_test <- preprocess_fn(x_test)
    }
    # nrounds, max_depth, eta, gamma, colsample_bytree, min_child_weight, subsample
    grid <- expand.grid(
        nrounds=1000,
        max_depth = c(32),
        eta = c(0.01),
        gamma = c(0.5),
        colsample_bytree = c(0.8),
        min_child_weight = c(1),
        subsample = c(0.4)
    )

    if(("Forb" %in% levels(train_labels)) && !("Forb"  %in% levels(test_labels))){
        levels(test_labels) <- c(levels(test_labels), "Forb")
        }
        
    train_ctrl <- caret::trainControl(
        method = "repeatedcv",
        number = 10,
        search = "grid",
        sampling = NULL,
        repeats = 3
    )
    
    pls_model <- caret::train(
        x_train, 
        train_labels, 
        #maxit = 100000,
        method="xgbTree",
        preProcess = c("center", "scale","knnImpute"),
        weights = weight_fn(train_labels),
        trControl = train_ctrl,
        tuneGrid = grid
        #tuneLength = n
    )

    print(pls_model)

    save(
        pls_model,
        file = file.path(save_path, paste0(model_id, ".rda"))
    )

    # create predictions (ranger)
    model_predictions <- predict(
            pls_model, 
            newdata = x_test
        )

    print(model_predictions)


    # generate the confusion matrix

    confusion_matrix <- caret::confusionMatrix(
        test_labels,
        model_predictions %>% as.factor(),
        mode = "everything"
    )

    log_model_results(
            model_id = model_id,
            confusion_matrix = confusion_matrix,
            custom = log_string,
            distribition = model_predictions %>% as.factor() %>% table(),
            logpath = "./gs_xgb.log")

    return(
        list(
                model = pls_model,
                confusion = confusion_matrix %>% as.list()
                )
    )

    

}

In [36]:
base_paths <- c(
    "img_raw_raw.csv",
    "grd_raw_raw.csv",
    "corrected_and_img.csv",
    "grd_corrected_only_bands.csv",# include veg indices
    "img_only_bands.csv",
    "grd_only_bands.csv",
    "corrected_and_img_only_bands.csv",
    "img_indices_only.csv",# include veg indices
    "grd_raw_corrected.csv",
    "grd_indices_only.csv"
)
# 

In [37]:
calculate_posterior_weights <- function(validation_path ="figures/merged_validation_s.csv" ){

    validation_df <- read.csv(validation_path, header = TRUE)
    #print(head(validation_df))

    total_observations <- sum(validation_df$validation_counts)
    #print(total_observations)
    weights <- (1/ validation_df$validation_prop)
    #print(validation_df$validation_prop)

    total_by_fg1 <- aggregate(
        x = validation_df$validation_counts,
        by = list(validation_df$key),
        FUN = sum
    )

    fg1_weight_list <- list()

    for( row_idx in seq(nrow(total_by_fg1))){
        name <- total_by_fg1$Group.1[[row_idx]]
        value <- total_by_fg1$x[[row_idx]]
        fg1_weight_list[name] <- value
    }
    
    return(fg1_weight_list)
}

get_posterior_weights_from_targets <- function(target_factor, posterior_weight = calculate_posterior_weights()){
    unbiased_weights <- targets_to_weights(target_factor)

    target_name_char <- target_factor %>% as.character()

    output_weights <- seq_along(target_factor)

    for(i in seq_along(target_factor)){
        if(posterior_weight[[target_name_char[[i]]]] > 0){
            fg1_weight <- 1 / posterior_weight[[target_name_char[[i]]]]
        } else {
            fg1_weight <- 0
        }
        output_weights[[i]] <- unbiased_weights[[i]] * fg1_weight
    }

    return(output_weights)
}

## Hypotheses

Peter
* Increasing number of trees will improve the accuracy/validation mismatch
* bias in 

Ken
* decreasing the number of the trees will decrease the impact of the posterior weighting on chi-squared statistic

In [38]:
outlier_functions <- list(
    #clip = load_model("./mle/clip_transform.rda"),
    no_treatment = function(x, ignore_cols = NULL){return(x)}# no transform
)

outlier_treatments <- c(
    "no_treatment",
    "clip"
)

preprocess_functions <- list(
    no_treatment = function(x, ignore_cols = NULL){return(x)},# no transform
    min_max = columnwise_min_max_scale,
    robust = columnwise_robust_scale,
    standard = standardize_df
)

weight_functions <- list(
    posterior = get_posterior_weights_from_targets,
    balanced = targets_to_weights,
    no_treatment = function(x){return(NULL)}# No weights
)
weight_treatments <- c(
    "balanced"#,
    #"no_treatment",
    #"posterior"
)

preprocessing_treatments <- c(
    "no_treatment"#,
    #"standard",
    #"min_max",
    #"robust"
)

In [39]:
num_components <- c(
    1,2,4,6,8,10,12,14,16,18,20,24,32,50#,64,75,100,128,200,256,512,1000
)

In [40]:
test_data <- subset(read.csv("Data/gs/x_test/img_raw_raw.csv"), select = -c(X))
test_labels <- read.csv("Data/gs/y_test/img_raw_raw.csv")$x %>% as.factor()
#train_labels <- read.csv("Data/gs/y_train/img_raw_raw.csv")$x %>% as.factor()
manifest_path <- "./gs_manifest_xgboost_fg0.csv"

In [41]:
for(filepath in base_paths){
    train_data <- subset(read.csv(paste0("Data/gs/x_train/", filepath)), select = -c(X))
    labels <- read.csv(paste0("Data/gs/y_train/", filepath))$x %>% as.factor()

                n <- "NA"
                p_treatment <- "no_treatment"
                o_treatment <- "no_treatment"
                w_treatment <- "no_treatment"
                print(p_treatment)
                print("Number of columns in train and test:")
                print(ncol(train_data))
                print(ncol(test_data))
                print("Number of rows in train and number of labels:")
                print(nrow(train_data))
                print(length(labels))

                model_id <- uuid::UUIDgenerate()
                save_path <- paste0("mle/experiments/gs/", model_id, "/")
                if(!dir.exists(save_path)){
                    dir.create(save_path)
                }

                rf_model_results <- train_pls_lda(
                    train_data, 
                    labels, 
                    test_data,
                    test_labels,
                    n = n,
                    outlier_fn = outlier_functions[[o_treatment]],
                    preprocess_fn = preprocess_functions[[p_treatment]],
                    weight_fn = weight_functions[[w_treatment]],
                    model_id = model_id,
                    seed=61718,
                    log_string = paste(n, filepath, o_treatment, p_treatment, w_treatment)
                )
                print(rf_model_results)

                rf_model <- rf_model_results$model
                acc <- as.list(rf_model_results$confusion$overall)$Accuracy
                print(acc)

                if(acc > 0.6){

                results <- validate_model(
                    rf_model, 
                    save_path, 
                    outlier_processing = outlier_functions[[o_treatment]],
                    transform_type = preprocess_functions[[p_treatment]],
                    pft_aggregation=0
                )

                aggregated_results <- aggregate_results(save_path)

                # calculate validation statistics
                chi2 <- calculate_chi_squared_probability(aggregated_results)
                r2 <- calculate_validation_r2(aggregated_results)
                rpd <- calculate_rpd(aggregated_results)


                add_model_to_manifest(
                    model_id = model_id,
                    outlier = o_treatment,
                    preprocessing = p_treatment,
                    source = filepath,
                    weight = w_treatment,
                    n = n,
                    oob_error = "NA",
                    accuracy = acc,
                    r2 = r2,
                    chi2prob = rpd,
                    logpath=manifest_path
                )

                plot_by_pft(
                    aggregated_results,
                    save_path = paste0(save_path, "aggregate.html"),
                    open = FALSE,
                    image_path = NULL,
                    aggregation=0
                )
            #
                write_validation_table(
                    aggregated_results,
                    save_path = paste0(save_path, "table.html"),
                    open = FALSE
                )
                } else {
                    add_model_to_manifest(
                        model_id = model_id,
                        outlier = o_treatment,
                        preprocessing = p_treatment,
                        source = filepath,
                        weight = w_treatment,
                        n = n,
                        oob_error = rf_model$prediction.error,
                        accuracy = acc,
                        r2 = "Skipped",
                        chi2prob = "Skipped",
                        logpath=manifest_path
                    )
                }
 }

[1] "no_treatment"
[1] "Number of columns in train and test:"
[1] 214
[1] 214
[1] "Number of rows in train and number of labels:"
[1] 4681
[1] 4681
eXtreme Gradient Boosting 

4681 samples
 214 predictor
   8 classes: 'Abiotic', 'Graminoid', 'Lichen', 'Moss', 'ShrubDecid', 'ShrubEvergreen', 'TreeBroadleaf', 'TreeConifer' 

Pre-processing: centered (214), scaled (214), nearest neighbor imputation (214) 
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 4213, 4214, 4213, 4213, 4212, 4213, ... 
Resampling results:

  Accuracy   Kappa    
  0.9789931  0.9599265

Tuning parameter 'nrounds' was held constant at a value of 1000

Tuning parameter 'min_child_weight' was held constant at a value of 1

Tuning parameter 'subsample' was held constant at a value of 0.4
  [1] Lichen         TreeConifer    TreeConifer    ShrubEvergreen TreeConifer   
  [6] TreeConifer    TreeConifer    TreeConifer    Abiotic        TreeConifer   
 [11] TreeConifer    TreeConifer    Gram

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.

`.cols` has been renamed and is deprecated, please use `.vars`


[1] "Handling Outliers with User supplied function"
[1] "Transforming Data with user supplied functions"


`.cols` has been renamed and is deprecated, please use `.vars`


Reading layer `Bisoon_Quadrats_georeferenced' from data source 
  `/home/krbundy/GitHub/lecospec/Data/Vectors/Bisoon_Quadrats_georeferenced.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 9 features and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -148.9524 ymin: 63.80698 xmax: -148.9508 ymax: 63.80701
Geodetic CRS:  WGS 84


...

Reading layer `Bonanza_N_Quads' from data source 
  `/home/krbundy/GitHub/lecospec/Data/Vectors/Bonanza_N_Quads.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 11 features and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: 438153.7 ymin: 7176051 xmax: 438205.6 ymax: 7176138
Projected CRS: WGS 84 / UTM zone 6N


Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Warning message in chisq.test(validation, prediction):
“Chi-squared approximation may be incorrect”


[1] "no_treatment"
[1] "Number of columns in train and test:"
[1] 119
[1] 214
[1] "Number of rows in train and number of labels:"
[1] 1343
[1] 1343
eXtreme Gradient Boosting 

1343 samples
 119 predictor
   9 classes: 'Abiotic', 'Forb', 'Graminoid', 'Lichen', 'Moss', 'ShrubDecid', 'ShrubEvergreen', 'TreeBroadleaf', 'TreeConifer' 

Pre-processing: centered (119), scaled (119), nearest neighbor imputation (119) 
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 1210, 1209, 1208, 1207, 1208, 1208, ... 
Resampling results:

  Accuracy   Kappa    
  0.8123304  0.7660998

Tuning parameter 'nrounds' was held constant at a value of 1000

Tuning parameter 'min_child_weight' was held constant at a value of 1

Tuning parameter 'subsample' was held constant at a value of 0.4
  [1] Lichen         Forb           Lichen         Lichen         Lichen        
  [6] Lichen         Graminoid      Graminoid      Abiotic        Lichen        
 [11] ShrubDecid     Graminoid  

Warning message in confusionMatrix.default(test_labels, model_predictions %>% as.factor(), :
“Levels are not in the same order for reference and data. Refactoring data to match.”


$model
eXtreme Gradient Boosting 

1343 samples
 119 predictor
   9 classes: 'Abiotic', 'Forb', 'Graminoid', 'Lichen', 'Moss', 'ShrubDecid', 'ShrubEvergreen', 'TreeBroadleaf', 'TreeConifer' 

Pre-processing: centered (119), scaled (119), nearest neighbor imputation (119) 
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 1210, 1209, 1208, 1207, 1208, 1208, ... 
Resampling results:

  Accuracy   Kappa    
  0.8123304  0.7660998

Tuning parameter 'nrounds' was held constant at a value of 1000

Tuning parameter 'min_child_weight' was held constant at a value of 1

Tuning parameter 'subsample' was held constant at a value of 0.4

$confusion
Confusion Matrix and Statistics

                Reference
Prediction       Abiotic Forb Graminoid Lichen Moss ShrubDecid ShrubEvergreen
  Abiotic             15    0         0      0    0          0              0
  Forb                 0    0         0      0    0          0              0
  Graminoid            0    9 

In [ ]:
sink(NULL)

In [ ]:
print(setdiff(colnames(test_data), colnames(train_data)))

In [ ]:
install.packages("rasterVis")

ERROR: Error: dbus: Call failed: [Errno 6] No such device or address: '/proc/22867/fd/2'
